In [35]:
import pandas as pd
import numpy as np

import os

In [36]:
BASE_DIR = "/Users/danielneves/00_DareData/090_pod_5/assignments/life_expectancy/tests/fixtures"

In [37]:
def _split_column(df_: pd.DataFrame) -> pd.DataFrame:
    """splits a column into several, based on a comma separating diff values"""
    df = df_.copy()
    new_cols = df['unit,sex,age,geo\\time'].str.split(',', expand=True)
    new_cols.columns = ['unit', 'sex', 'age', 'region']
    df = pd.concat([df, new_cols], axis=1)
    df = df.drop(['unit,sex,age,geo\\time'], axis=1)
    return df

In [38]:
def clean_data(df_: pd.DataFrame, region: str = "PT") -> None:
    """ receives a dataframe and do some cleaning"""

    df = df_.copy()
    df = _split_column(df)
    df = df.melt(id_vars=["unit", "sex", "age", "region"], var_name="year", value_name="value")
    df = extract_numeric_values_from_column(df, "value")
    df = df.dropna(subset=["value"])
    df["year"] = df["year"].astype(int)
    df["value"] = df["value"].astype(float)
    df = df[df.region == region]
    return df

In [39]:
def extract_numeric_values_from_column(df_: pd.DataFrame, column: str) -> pd.DataFrame:
    """extracts numerical values from a string including several strange characteres"""
    df = df_.copy()
    df[column]= df[column].replace(to_replace=r'[^\d\.]+', value='', regex=True)
    df[column] = df[column].replace(to_replace='', value=np.nan)
    df[column] = pd.to_numeric(df[column], errors='coerce')
    return df

In [40]:
FILE_DIR = "/Users/danielneves/00_DareData/090_pod_5/assignments/life_expectancy/data"
FIXTURE_DIR = "/Users/danielneves/00_DareData/090_pod_5/assignments/life_expectancy/tests/fixtures"

In [41]:
from pathlib import Path
import pandas as pd

FILE_DIR = Path("/Users/danielneves/00_DareData/090_pod_5/assignments/life_expectancy/data")
df = pd.read_csv(FILE_DIR.joinpath("eu_life_expectancy_raw.tsv"), sep='\t')
df_cleaned = clean_data(df, region="PT")


In [63]:
df = pd.read_csv(os.path.join(FIXTURE_DIR, 'eu_life_expectancy_raw.tsv'), delimiter="\t")

In [64]:
df

,"unit,sex,age,geo\time",2021,2020,2019,2018,2017,2016,2015,2014,2013,...,1969,1968,1967,1966,1965,1964,1963,1962,1961,1960
0,"YR,F,Y1,AL",:,79.4,80.4,80.2,79.7,79.8,79.2,79.8,79.6,...,:,:,:,:,:,:,:,:,:,:
1,"YR,F,Y1,AM",:,:,79.1,79.2,78.5,78.0,77.9,:,:,...,:,:,:,:,:,:,:,:,:,:
2,"YR,F,Y1,AT",:,82.9,83.5,83.3,83.2,83.4,83.0,83.3,83.0,...,:,:,:,:,:,:,:,:,:,:
3,"YR,F,Y1,AZ",:,:,78.6,78.1,77.7,:,77.5,77.0,76.9,...,:,:,:,:,:,:,:,:,:,:
4,"YR,F,Y1,BE",:,82.3,83.6,83.2,83.2,83.2,82.6,83.1,82.5,...,74.4,74.3,74.7,74.4,74.3,74.6,73.9,74.0,74.5,73.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14443,"YR,T,Y_LT1,SM",:,:,:,:,:,:,:,:,:,...,:,:,:,:,:,:,:,:,:,:
14444,"YR,T,Y_LT1,TR",:,:,79.1,78.9,78.5,78.1,78.2,78.1,78.2,...,:,:,:,:,:,:,:,:,:,:
14445,"YR,T,Y_LT1,UA",:,:,73.4,73.2,73.3,72.9,72.5,71.8,:,...,:,:,:,:,:,:,:,:,:,:
14446,"YR,T,Y_LT1,UK",:,:,:,81.3,81.3,81.2,81.0,81.4,81.1,...,:,:,:,:,:,:,:,:,:,:


In [68]:
df_cleaned = clean_data(df, 'PT')

In [69]:
df_cleaned

,unit,sex,age,region,year,value
3460,YR,F,Y65,PT,2021,21.7
4804,YR,F,Y_LT1,PT,2021,84.3
8276,YR,M,Y65,PT,2021,17.8
9620,YR,M,Y_LT1,PT,2021,78.0
13092,YR,T,Y65,PT,2021,19.9
...,...,...,...,...,...,...
895540,YR,T,Y83,PT,1960,4.7
895596,YR,T,Y84,PT,1960,4.5
895652,YR,T,Y9,PT,1960,62.9
895708,YR,T,Y_GE85,PT,1960,4.2


In [76]:
df_cleaned.to_csv(os.path.join(FIXTURE_DIR, "pt_life_expectancy_expected.csv"), index = True)

In [79]:
df_expected = pd.read_csv(os.path.join(FIXTURE_DIR, 'pt_life_expectancy_expected.csv'),index_col=0)

In [80]:
df_expected

,unit,sex,age,region,year,value
3460,YR,F,Y65,PT,2021,21.7
4804,YR,F,Y_LT1,PT,2021,84.3
8276,YR,M,Y65,PT,2021,17.8
9620,YR,M,Y_LT1,PT,2021,78.0
13092,YR,T,Y65,PT,2021,19.9
...,...,...,...,...,...,...
895540,YR,T,Y83,PT,1960,4.7
895596,YR,T,Y84,PT,1960,4.5
895652,YR,T,Y9,PT,1960,62.9
895708,YR,T,Y_GE85,PT,1960,4.2
